In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By


options = Options()
options.add_argument("--headless")   # <— force headless mode

service = Service()  # or Service("path/to/geckodriver") if not in PATH
driver = webdriver.Firefox(service=service, options=options)


try:
    # Go to the target URL
    driver.get("https://www.macfit.com/kulupler")

    # Locate the dropdown
    select_element = driver.find_element(By.TAG_NAME, "select")

    # Grab all option elements
    options_list = select_element.find_elements(By.TAG_NAME, "option")

    # Extract city links (skip the first "Tüm Şehirler" entry)
    city_links = [opt.get_attribute("value") for opt in options_list if "kulupler/" in opt.get_attribute("value")]

    # Print result
    for link in city_links:
        print(link)

finally:
    driver.quit()


https://www.macfit.com/kulupler/adana
https://www.macfit.com/kulupler/ankara
https://www.macfit.com/kulupler/antalya
https://www.macfit.com/kulupler/bursa
https://www.macfit.com/kulupler/denizli
https://www.macfit.com/kulupler/diyarbakir
https://www.macfit.com/kulupler/edirne
https://www.macfit.com/kulupler/eskisehir
https://www.macfit.com/kulupler/gaziantep
https://www.macfit.com/kulupler/istanbul
https://www.macfit.com/kulupler/izmir
https://www.macfit.com/kulupler/kocaeli
https://www.macfit.com/kulupler/manisa
https://www.macfit.com/kulupler/mersin
https://www.macfit.com/kulupler/mugla
https://www.macfit.com/kulupler/sakarya
https://www.macfit.com/kulupler/samsun


In [4]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time

# --- Setup Firefox in headless mode ---
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

# city_links is already available from phase 1
city_links = [
    "https://www.macfit.com/kulupler/adana",
    # "https://www.macfit.com/kulupler/ankara",
    "https://www.macfit.com/kulupler/antalya",
    # ... all other cities
]

# Dictionary: {city_name: [branch_links]}
gyms_by_city = {}

try:
    for city_url in city_links:
        driver.get(city_url)
        time.sleep(2)  # let the page load

        # Get city name from URL
        city_name = city_url.rstrip("/").split("/")[-1]

        # Find all branch links in that city
        branch_cards = driver.find_elements(
            By.CSS_SELECTOR, "div.our-informations-grid-card.branchitem a.grid-image"
        )
        branch_links = [card.get_attribute("href") for card in branch_cards]

        gyms_by_city[city_name] = branch_links
        print(f"{city_name}: {len(branch_links)} gyms found")

finally:
    driver.quit()

# Show collected results
for city, links in gyms_by_city.items():
    print(f"\nCity: {city}")
    for link in links:
        print("  ", link)


adana: 2 gyms found
antalya: 3 gyms found

City: adana
   https://www.macfit.com/kulupler/adana/seyhan/macfit-01-burda
   https://www.macfit.com/kulupler/adana/yuregir/macfit-adana-optimum

City: antalya
   https://www.macfit.com/kulupler/antalya/kepez/macfit-antalya-erasta
   https://www.macfit.com/kulupler/antalya/muratpasa/macfit-antalya-lara-carrefour
   https://www.macfit.com/kulupler/antalya/muratpasa/macfit-markantalya


In [5]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

# --- Setup Firefox in headless mode ---
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

# Example gyms_by_city dict (from previous step)
# gyms_by_city = {
#     "adana": [
#         "https://www.macfit.com/kulupler/adana/seyhan/macfit-01-burda",
#         "https://www.macfit.com/kulupler/adana/seyhan/macfit-m1"
#     ],
#     "ankara": [
#         "https://www.macfit.com/kulupler/ankara/cankaya/macfit-next-level"
#     ]
#     # ... add all other cities
# }

# Results list
data = []

try:
    for city, gym_links in gyms_by_city.items():
        for gym_url in gym_links:
            driver.get(gym_url)
            time.sleep(2)  # wait for page to load

            try:
                # Extract address (if exists)
                address_el = driver.find_element(By.CSS_SELECTOR, "p.contact-detail-content")
                address = address_el.text.strip()
            except:
                address = None  # If no address found

            data.append({
                "City": city,
                "Gym_URL": gym_url,
                "Address": address
            })
            print(f"Scraped {city} → {gym_url} → {address}")

finally:
    driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV & Excel
df.to_csv("macfit_gyms.csv", index=False, encoding="utf-8-sig")
df.to_excel("macfit_gyms.xlsx", index=False)

print("\n✅ Data saved to macfit_gyms.csv and macfit_gyms.xlsx")


Scraped adana → https://www.macfit.com/kulupler/adana/seyhan/macfit-01-burda → Ahmet Remzi Yüreğir Mah., Turhan Cemal Beriker Blv. 162-1 Seyhan/Adana
Scraped adana → https://www.macfit.com/kulupler/adana/yuregir/macfit-adana-optimum → Yeşilyurt Mah., 70028. Sok., 01150 Yüreğir/Adana
Scraped antalya → https://www.macfit.com/kulupler/antalya/kepez/macfit-antalya-erasta → Erasta AVM Ahatlı Mah., Dumlupınar Blv., Yanyolu No:25, 07090 Kepez/Antalya
Scraped antalya → https://www.macfit.com/kulupler/antalya/muratpasa/macfit-antalya-lara-carrefour → Şirinyalı Mah. İsmet Gökşen Cad. Carrefoursa Avm No:5 Muratpaşa/Antalya
Scraped antalya → https://www.macfit.com/kulupler/antalya/muratpasa/macfit-markantalya → Tahılpazarı Mah. Kazım Özalp Cad. (Şarampol) No:84 Muratpaşa/Antalya

✅ Data saved to macfit_gyms.csv and macfit_gyms.xlsx
